In [42]:
from plyfile import PlyData, PlyElement
from glob import glob
from utils.pointcloud import grid_subsample_simple
import os
from omegaconf import OmegaConf
import shutil
import pandas as pd
from scipy.spatial import KDTree
import numpy as np
import json





In [43]:
cfg = OmegaConf.merge(
    OmegaConf.load("./config/default.yaml"),
    OmegaConf.load("./config/subsampling.yaml"),
    OmegaConf.load("./config/label_merge.yaml"))


in_dir = cfg.data.preprocess_raw.output_dir_subsampling
out_dir = cfg.data.preprocess_raw.output_dir_merged

In [47]:
# build mapping
with open(os.path.join(in_dir,"class_dict.json"),'r') as f: # get input schema
    label_schema_old = json.load(f)
mapping = dict()
for label_level in label_schema_old.keys():
    mapping[label_level] = dict()
    for label_name in label_schema_old[label_level].keys():
        
        if cfg.data.label_merge.groups[label_level] and label_name in [v for vs in cfg.data.label_merge.groups[label_level].values() for v in vs]:
            k = [k for k,v in cfg.data.label_merge.groups[label_level].items() if label_name in v][0]
            mapping[label_level][label_name] = k
        else:
            mapping[label_level][label_name] = label_name
        

# build new schema
label_schema_new = dict()
for label_level in mapping.keys():
    new_label_names = np.unique(list(mapping[label_level].values()))
    label_schema_new[label_level]={v:k for k,v in enumerate(new_label_names)}

# build nummeric mapping for efficient indexing
mapping_nummeric = dict()
for label_level in label_schema_old.keys():
    mapping_nummeric[label_level]=dict()
    for label_name_old, label_idx_old in label_schema_old[label_level].items():
        mapping_nummeric[label_level][label_idx_old] = label_schema_new[label_level][mapping[label_level][label_name_old]]
    mapping_array = np.ones(len(mapping_nummeric[label_level]))*-1
    for o,n in mapping_nummeric[label_level].items():
        mapping_array[o] = n
    mapping_nummeric[label_level] = mapping_array
    assert np.all(mapping_array >= 0), "Some elements couldn't be mapped correctly"

print(label_schema_new)

for proj_fp in glob(os.path.join(in_dir, "*.FwfProj")):
    project_name = os.path.basename(proj_fp)
    print(f"{project_name}")

    pcd_fp = list(glob(os.path.join(proj_fp,'labeled',"*_pointcloud.ply")))[0]
    waveform_fp = list(glob(os.path.join(proj_fp,'labeled',"*_waveform.npy")))[0]
    meta_fp = waveform = list(glob(os.path.join(proj_fp,'labeled',"*_metadata.json")))[0]

    pcd = pd.DataFrame.from_dict(PlyData.read(pcd_fp).elements[0].data)
    
    
    for label_level in label_schema_old.keys():
        pcd[label_level] = mapping_nummeric[label_level][pcd[label_level]].astype(np.uint8)
    
    proj_out_dir = os.path.join(out_dir,project_name,'labeled')
    os.makedirs(proj_out_dir)

    PlyData([PlyElement.describe(pcd.to_records(index=False),'vertex')]).write(os.path.join(proj_out_dir, os.path.basename(pcd_fp)))
    shutil.copy(waveform_fp,os.path.join(proj_out_dir,os.path.basename(waveform_fp)))
    shutil.copy(meta_fp,os.path.join(proj_out_dir,os.path.basename(meta_fp)))


with open(os.path.join(out_dir,"class_dict.json"),'w') as f:
    json.dump(label_schema_new, f)


{'labels_0': {'manMade': 0, 'natural': 1, 'scanArtefact': 2}, 'labels_1': {'_unspecified': 0, 'building': 1, 'movingObject': 2, 'naturalGround': 3, 'sealedSurface': 4, 'streetFurniture': 5, 'vegetation': 6, 'vehicle': 7}, 'labels_2': {'_unspecified': 0, 'bicycle': 1, 'car': 2, 'electricalAppliance': 3, 'facadeSurface': 4, 'fence': 5, 'frameElement': 6, 'kerbStone': 7, 'naturalGround': 8, 'nonTiledPaving': 9, 'pipeLike': 10, 'scooter': 11, 'shaft': 12, 'sign': 13, 'stair': 14, 'tiledPaving': 15, 'trashCan': 16, 'vegetation': 17}, 'labels_3': {'_unspecified': 0, 'asphalt': 1, 'brick': 2, 'cable': 3, 'concrete': 4, 'marking': 5, 'mesh': 6, 'metal': 7, 'naturalStone': 8, 'poster': 9, 'treeTrunk': 10, 'vegetation': 11}}
2023-08-28_FW_EingangBauing.FwfProj
2024-03-22_FW_Koenigshuegel.FwfProj
2024-04-05_FW_Westbahnhof_01.FwfProj
2024-04-05_FW_Westbahnhof_02.FwfProj
2024-04-05_FW_Westbahnhof_03.FwfProj
2024-04-05_FW_Westbahnhof_04.FwfProj
2024-04-05_FW_Westbahnhof_05.FwfProj
2024-05-10_FW_RWTH